In [1]:
import pandas as pd
import numpy as np

In [2]:
# df = pd.read_excel('../table-a2-annual-building-activity-report-summary-new-construction-entitled-permits-and-completed-units.xlsx')
df = pd.read_excel('/Users/sidharth.kapur/Downloads/table-a2-annual-building-activity-report-summary-new-construction-entitled-permits-and-completed-units.xlsx')

In [3]:
totals = (
    df
    .groupby(['Jurisidiction', 'County', 'Reporting Year'])['Total\xa0 Units Permitted']
    .sum()
    .unstack()
    .reset_index()
)

In [5]:
totals

Reporting Year,Jurisidiction,County,2018,2019,2020,2021
0,ADELANTO,SAN BERNARDINO,27.0,63.0,NaN,NaN
1,AGOURA HILLS,LOS ANGELES,27.0,12.0,3.0,8.0
2,ALAMEDA,ALAMEDA,181.0,681.0,298.0,605.0
3,ALAMEDA COUNTY,ALAMEDA,119.0,89.0,63.0,NaN
4,ALBANY,ALAMEDA,17.0,31.0,36.0,NaN
...,...,...,...,...,...,...
514,YREKA,SISKIYOU,1.0,1.0,1.0,NaN
515,YUBA CITY,SUTTER,32.0,32.0,61.0,NaN
516,YUBA COUNTY,YUBA,422.0,762.0,742.0,NaN
517,YUCAIPA,SAN BERNARDINO,110.0,42.0,37.0,NaN


In [35]:
goals = pd.read_excel('ABAG_Final_2023-2031_RHNA_Allocation.xlsx', skiprows=1, header=2)
goals = goals.dropna(subset=['TOTAL'])
goals = goals[
    goals['Jurisdiction'] != 'TOTAL'
]
goals = goals[[
    'Jurisdiction', 'TOTAL'
]]
goals['Jurisdiction'] = goals['Jurisdiction'].str.upper()
goals.loc[
    goals['Jurisdiction'] == 'SAN FRANCISCO (CITY)', 'Jurisdiction'
] = "SAN FRANCISCO"

In [36]:
goals

,Jurisdiction,TOTAL
1,ALAMEDA,5353.0
2,ALBANY,1114.0
3,BERKELEY,8934.0
4,DUBLIN,3719.0
5,EMERYVILLE,1815.0
...,...,...
117,SANTA ROSA,4685.0
118,SEBASTOPOL,213.0
119,SONOMA,311.0
120,UNINCORPORATED SONOMA,3881.0


In [37]:
totals_bay_area = totals.copy().rename(
    columns={
        'Jurisidiction': 'Jurisdiction'
    }
).rename_axis(columns=None)

In [38]:
totals_bay_area['Average rate'] = totals_bay_area[[2018, 2019, 2020, 2021]].mean(axis=1)
totals_bay_area['8-year rate'] = 8 * totals_bay_area['Average rate']

In [39]:
joined = goals.rename(
    columns={
        'TOTAL': '6th cycle RHNA'
    }
).merge(totals_bay_area, on='Jurisdiction', how='left')
joined['fraction'] = joined['8-year rate'] / joined['6th cycle RHNA']
joined['Jurisdiction'] = joined['Jurisdiction'].str.title()
joined['County'] = joined['County'].str.title()
joined['6th cycle RHNA'] = joined['6th cycle RHNA'].astype('Int64')

joined['8-year rate'] = joined['8-year rate'].round(1)
joined['fraction'] = joined['fraction'].round(2)

joined[2018] = joined[2018].astype('Int64')
joined[2019] = joined[2019].astype('Int64')
joined[2020] = joined[2020].astype('Int64')
joined[2021] = joined[2021].astype('Int64')

In [42]:
joined

,Jurisdiction,6th cycle RHNA,County,2018,2019,2020,2021,Average rate,8-year rate,fraction
0,Alameda,5353,Alameda,181,681,298,605,441.250000,3530.0,0.66
1,Albany,1114,Alameda,17,31,36,<NA>,28.000000,224.0,0.20
2,Berkeley,8934,Alameda,340,628,976,<NA>,648.000000,5184.0,0.58
3,Dublin,3719,Alameda,787,241,500,777,576.250000,4610.0,1.24
4,Emeryville,1815,Alameda,78,0,503,252,208.250000,1666.0,0.92
...,...,...,...,...,...,...,...,...,...,...
97,Santa Rosa,4685,Sonoma,457,447,623,1404,732.750000,5862.0,1.25
98,Sebastopol,213,Sonoma,13,5,27,7,13.000000,104.0,0.49
99,Sonoma,311,Sonoma,24,105,125,<NA>,84.666667,677.3,2.18
100,Unincorporated Sonoma,3881,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [43]:
joined[
    joined['fraction'] >= 1
].sort_values('fraction', ascending=False).reset_index(drop=True)

,Jurisdiction,6th cycle RHNA,County,2018,2019,2020,2021,Average rate,8-year rate,fraction
0,Brentwood,1522,Contra Costa,345,688,983,<NA>,672.000000,5376.0,3.53
1,Oakley,1058,Contra Costa,184,262,467,490,350.750000,2806.0,2.65
2,American Canyon,446,Napa,162,1,5,353,130.250000,1042.0,2.34
3,Sonoma,311,Sonoma,24,105,125,<NA>,84.666667,677.3,2.18
4,Rohnert Park,1580,Sonoma,594,313,232,<NA>,379.666667,3037.3,1.92
5,Morgan Hill,1037,Santa Clara,519,167,44,<NA>,243.333333,1946.7,1.88
6,Newark,1874,Alameda,460,279,284,<NA>,341.000000,2728.0,1.46
7,Colma,202,San Mateo,69,0,<NA>,<NA>,34.500000,276.0,1.37
8,Los Altos Hills,489,Santa Clara,28,47,87,149,77.750000,622.0,1.27
9,Santa Rosa,4685,Sonoma,457,447,623,1404,732.750000,5862.0,1.25


In [44]:
joined.to_csv('Permits in 2018-2021 vs. 6th cycle RHNA.csv')